# Construcción de una aplicación web de recomendación de cocina

En esta lección, construirás un modelo de clasificación usando algunas de las técnicas que aprendiste en las lecciones anteriores y con el conjunto de datos de la cocina deliciosa usada a través de este serie de lecciones. Además, construirás una pequeña aplicación web para usar un modelo guardado, aprovechando el runtime web de Onnx.

Uno de los usos prácticos más útiles del aprendizaje automático es construir sistemas de recomendación, y ¡hoy puedes tomar el primer en esa dirección!

En esta lección aprenderás:

* Cómo construir un modelo y guardarlo como un modelo Onnx
* Cómo usar Netron para inspeccionar el modelo
* Cómo usar tu modelo en una aplicación web por inferencia

## Construye tu modelo
Construir sistemas de aprendizaje automático aplicado es un parte importante de aprovechar estas tecnologías para tus sistemas de negocio. Puedes usar modelos dentro de tus aplicaciones web (y así usarlos de sin conexión en caso de ser necesario) al usar Onnx.

En la lección anterior, construiste un modelo de regresión acerca de los avistamientos OVNI, le hiciste "pickle", y los usaste en una aplicación Flask. Aunque esta arquitectura es muy útil conocerla, es una aplicación Python full-stack, y tus requerimientos pueden incluir el uso de una aplicación JavaScript.

En esta lección, puedes construir un sistema JavaScript básico por inferencia. Pero primero, necesitas entrenar tu modelo y convertirlo para usarlo con Onnx.

### Ejercicio - entrena tu modelo de clasificación
Primero, entrena un modelo de clasificación usando el conjunto limpio de datos de cocina que ya usamos.

1. Comienza importando bibliotecas útiles:

In [1]:
%pip install skl2onnx
import pandas as pd 

Necesitas 'skl2onnx' para ayudar a convertir tu modelo Scikit-learn al formato Onnx.

2. Luego, trabaja con tus datos de la misma forma que lo hiciste en las lecciones anteriores, al leer el archivo CSV usando read_csv():

In [2]:
data = pd.read_csv('../data/cleaned_cuisines.csv')
data.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


3. Elimina las primeras dos columnas y guarda los datos restantes como 'X':

In [3]:
X = data.iloc[:,2:]
X.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


4. Guarda las etiquetas como 'y':

In [4]:
y = data[['cuisine']]
y.head()

,cuisine
0,indian
1,indian
2,indian
3,indian
4,indian


### Comienza la rutina de entrenamiento
Usaremos la biblioteca 'SVC' la cual tiene buena precisión.

1. Importa las bibliotecas correspondientes de Scikit-learn:

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report

2. Separa los conjuntos de entrenamiento y prueba:

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

3. Construye un model de clasificación SVC como lo hiciste en la lección anterior:

In [7]:
model = SVC(kernel='linear', C=10, probability=True,random_state=0)
model.fit(X_train,y_train.values.ravel())

SVC(C=10, kernel='linear', probability=True, random_state=0)

4. Ahora, prueba tu modelo al llamar a predict():

In [8]:
y_pred = model.predict(X_test)

5. Imprime un reporte de clasificación para revisar la calidad del modelo:

In [9]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.62      0.71      0.66       231
      indian       0.89      0.87      0.88       234
    japanese       0.79      0.75      0.77       248
      korean       0.80      0.71      0.76       234
        thai       0.80      0.82      0.81       252

    accuracy                           0.77      1199
   macro avg       0.78      0.77      0.78      1199
weighted avg       0.78      0.77      0.78      1199



### Convierte tu modelo a Onnx
Asegúrate que haces la conversión con el número adecuado de Tensor. Este conjunto de datos lista 380 ingredientes, por lo que necesitas anotar ese número en FloatTensorType:

1. Conviértelo usando un número de tensor de 380.

In [10]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}

2. Crea el onx y guárdalo como un archivo model.onnx:

In [11]:
onx = convert_sklearn(model, initial_types=initial_type, options=options)
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())

Despues de crear el modelo onx y construir la aplicacion web, ejecuta en consola el comando: `npx http-server`